In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import cPickle as pickle
from glob import glob
import os
import random
import numpy as np

import threebody

In [3]:
bpd = {}

In [4]:
bpd = threebody.load_best_parameter_database()

### Update the database based on fitter results

In [39]:
d = pickle.load(open("fitter_params.pickle"))
F = threebody.Fitter(**d)
p = pickle.load(open("fitter_best_values.pickle"))
newp = F.best_parameters.copy()

newp.update(p)
old = -2*F.efac**2*(F.lnprob(F.best_parameters)+F.lnprior(F.best_parameters))
new = -2*F.efac**2*(F.lnprob(newp)+F.lnprior(newp))
print "existing best parameters:", old
print "best parameters from fitter:", new

existing best parameters: 33794.2325396
best parameters from fitter: 33794.2325396


In [38]:
if new<old:
    bpd[frozenset(F.args.iteritems())] = newp
threebody.save_best_parameter_database(bpd)

### Update the database by looking at the best results from an MCMC run

In [40]:
jobid = "69543"
job_dir = glob(os.path.join("emcee-chains",jobid+".*"))[0]

In [41]:
with open(os.path.join(job_dir,"fitter_params.pickle")) as f:
    fitter_params = pickle.load(f)
F = threebody.Fitter(**fitter_params)

In [42]:
best_so_far = np.load(os.path.join(job_dir,"best_so_far.npy"))
newp = F.best_parameters.copy()
for i,p in enumerate(F.parameters):
    newp[p] = best_so_far[i]

In [43]:
old = -2*F.efac**2*(F.lnprob(F.best_parameters)+F.lnprior(F.best_parameters))
new = -2*F.efac**2*(F.lnprob(newp)+F.lnprior(newp))
print "existing best parameters:", old
print "best parameters from MCMC:", new

existing best parameters: 33794.2325396
best parameters from MCMC: 33797.3887062


In [44]:
if new<old:
    bpd[frozenset(F.args.iteritems())] = newp
threebody.save_best_parameter_database(bpd)

### Update the database for a fitter by trying values for all known configurations

In [ ]:
#d = pickle.load(open("fitter_params.pickle"))
#d = pickle.load(open("emcee-chains/32534.nimrod/fitter_params.pickle"))
#d = pickle.load(open("fake_toas_fitter.pickle"))
d = pickle.load(open("forecaster_params.pickle"))
#d['files'] = 'fake-uniform'
#d['kopeikin'] = True
#d['fit_pm'] = True
#d['fit_px'] = True

#d = pickle.load(open("sensitivity_params.pickle"))
#d = pickle.load(open("plot_params.pickle"))
#d = pickle.load(open("scratch_params.pickle"))
print d
F = threebody.Fitter(**d)
if "j_NCY1400" not in F.best_parameters:
    F.best_parameters["j_NCY1400"] = 0.25*2.7e-3
if "delta" not in F.best_parameters:
    F.best_parameters['delta'] = 0
try:
    old = -2*F.efac**2*(F.lnprob(F.best_parameters)+F.lnprior(F.best_parameters))
except KeyError:
    print "Missing best parameter values!"
    old = np.inf
print "existing best parameters:", old
best = old
best_p = F.best_parameters.copy()
itms = bpd.items()
random.shuffle(itms)
for k, p in itms: 
    newp = F.best_parameters.copy()
    newp.update(p)
    new = -2*F.efac**2*(F.lnprob(newp)+F.lnprior(newp))
    print "best parameters from database:", new
    if new<best:
        best = new
        best_p = newp
        if True:
            bpd[frozenset(F.args.iteritems())] = best_p
            threebody.save_best_parameter_database(bpd)


{'ppn_mode': 'heavysimple', 'tzrmjd_middle': False, 'priors': ('dbeta', 'dgamma'), 't2_astrometry': True, 'fit_pm': False, 'files': '0337+17-anne-2014-09-22', 'kopeikin': False, 'efac': (('AO1440', 1.45), ('GBT1500', 1.11), ('WSRT1400', 1.45)), 'parfile': '0337_tempo2_px_optical.par', 'fit_pos': True, 'fit_px': False}


No handlers could be found for logger "threebody"


Missing best parameter values!
existing best parameters: inf
best parameters from database: 1355683854.61
best parameters from database: 59101385.0785
best parameters from database: 34392.5868496
best parameters from database: 34392.5868496
best parameters from database: 59063213.0186
best parameters from database: 59262248.9467
best parameters from database: 59262248.9467
best parameters from database: 59026112.6342
best parameters from database: 59101385.0785
best parameters from database: 34392.5868496
best parameters from database: 59063213.0186
best parameters from database: 6974270293.04
best parameters from database: 59026112.6342
best parameters from database: 59262248.9467
best parameters from database: 59026112.6342
best parameters from database: 122134.241504
best parameters from database: 35459.8316201
best parameters from database: 59156893.7861
best parameters from database: 59096633.8386
best parameters from database: 6994553656.76
best parameters from database: 59101385

In [8]:
best_p

{'acosi_i': 1.4917920737881377967,
 'acosi_o': 91.515517821405981132,
 'asini_i': 1.2175268187146886046,
 'asini_o': 74.672709901274213484,
 'd_DECJ': 3.8297379260835209625e-07,
 'd_RAJ': -9.1473641598984196357e-08,
 'dbeta': 3.137215781113322372e-05,
 'delta': -2.4326288381809617149e-10,
 'delta_lan': -5.4096316391090949558e-05,
 'dgamma': -2.5854449182822343408e-07,
 'eps1_i': 0.00068724728071223587401,
 'eps1_o': 0.035186247652040840626,
 'eps2_i': -9.1179836780626676656e-05,
 'eps2_o': -0.003462135260726911092,
 'f0': 365.95336311711874974,
 'f1': -2.3547358099625887817e-15,
 'j_AO1350': 5.364017648361716e-05,
 'j_AO1440': -0.00055121663140921818248,
 'j_AO327': 6.457616019511097e-05,
 'j_GBT1500': -0.00053763724410558742536,
 'j_GBT350': 1.8912917353649655e-05,
 'j_GBT820': 6.712253154451129e-05,
 'j_NCY1400': 0.000675,
 'j_WSRT350': -3.6063906052781444e-05,
 'pb_i': 1.6293969394546642386,
 'pb_o': 327.2575252205244358,
 'ppn_mode': 'heavysimple',
 'q_i': 0.13746673219074406775,
 

In [9]:
bpd[frozenset(F.args.iteritems())] = best_p
threebody.save_best_parameter_database(bpd)

In [10]:
bpd.items()

[(frozenset({('efac', 1.3),
             ('files', '0337+17-anne-2014-09-22'),
             ('fit_pm', False),
             ('fit_pos', True),
             ('fit_px', False),
             ('general', True),
             ('kopeikin', False),
             ('matrix_mode', 0),
             ('only_tels', ('GBT1500', 'WSRT1400')),
             ('parfile', '0337_tempo2_px_optical.par'),
             ('ppn_mode', 'heavysimple'),
             ('priors', ('dbeta', 'dgamma')),
             ('shapiro', True),
             ('special', True),
             ('t2_astrometry', True),
             ('tol', 1e-16),
             ('tzrmjd_middle', 'auto'),
             ('use_quad', False)}),
  {'acosi_i': 1.4920602002505874639,
   'acosi_o': 91.545730948517159101,
   'asini_i': 1.2175272341154308563,
   'asini_o': 74.672712356463980574,
   'd_DECJ': 4.4265935417303728307e-07,
   'd_RAJ': -1.070284187936875009e-07,
   'dbeta': -0.0019127355350863758238,
   'delta': -1.1740885304932356904e-10,
   'delta_lan': 

In [8]:
newp

{'acosi_i': 1.4897526614420296516,
 'acosi_o': 91.406877920363995858,
 'asini_i': 1.2175284370901029975,
 'asini_o': 74.672709712018036338,
 'delta': 0.0,
 'delta_lan': 4.8220241628483202396e-05,
 'eps1_i': 0.0006856734568106118968,
 'eps1_o': 0.035186277265651086594,
 'eps2_i': -9.1706450063988849455e-05,
 'eps2_o': -0.0034621368067846675071,
 'f0': 365.95336876828094,
 'j_AO1350': 5.3635190284990956237e-05,
 'j_AO1440': 4.9259452945711710099e-05,
 'j_AO327': 6.4794561335151349273e-05,
 'j_GBT1500': 6.2620813306723303735e-05,
 'j_GBT350': 1.8855899287062338046e-05,
 'j_GBT820': 6.7057259469771009361e-05,
 'j_WSRT350': -3.6104142237134380419e-05,
 'pb_i': 1.6294017912103802948,
 'pb_o': 327.25753790301990501,
 'ppn_mode': None,
 'q_i': 0.13734741974750966142,
 'shapiro': True,
 'tasc_i': 0.40771744400081470636,
 'tasc_o': 313.93581137879523948,
 'tol': 1e-16,
 'use_quad': False}

In [9]:
for k in bpd.keys():
    d = dict(k)
    if False and 'shapiro' not in d:
        d['shapiro'] = True
        bpd[frozenset(d.iteritems())] = bpd[k]
        d['shapiro'] = False
        bpd[frozenset(d.iteritems())] = bpd[k]
    if d['parfile'] == '0337_tempo2_pm.par':
        d['parfile'] = '0337_tempo2_px.par'
        nk = frozenset(d.iteritems())
        if nk not in bpd:
            bpd[nk] = bpd[k]

In [102]:
for k in F.best_parameters:
    if k not in p:
        print "missing from p:", k, F.best_parameters[k]
    elif F.best_parameters[k]!=p[k]:
        print "different:", k, F.best_parameters[k], p[k]
        

missing from p: f0 365.953368768


In [94]:
for k in bpd.keys():
    if k==frozenset(d.iteritems()):
        bpd[k].pop('pm_x',None)
        bpd[k].pop('pm_y',None)
        bpd[k].pop('f1',None)

In [107]:
F.parameters

['asini_i',
 'pb_i',
 'eps1_i',
 'eps2_i',
 'tasc_i',
 'acosi_i',
 'q_i',
 'asini_o',
 'pb_o',
 'eps1_o',
 'eps2_o',
 'tasc_o',
 'acosi_o',
 'delta_lan',
 'lan',
 'delta',
 'dgamma',
 'dbeta',
 'd_RAJ',
 'd_DECJ',
 'd_PX',
 'j_AO1350',
 'j_AO1440',
 'j_AO327',
 'j_GBT1500',
 'j_GBT350',
 'j_GBT820',
 'j_WSRT350']

In [6]:
paper1 = dict(files="0337+17-scott-2013-06-06",
                         parfile="0337_tempo2_nobinary.par",
                         tzrmjd_middle='auto',
                         fit_pos=False,
                         fit_pm=False,
                         fit_px=False,
                         t2_astrometry=True,
                         kopeikin=False,
                         priors=(),
                         ppn_mode=None)

In [11]:
F = threebody.Fitter(**paper1)

In [13]:
bpd[frozenset(F.args.iteritems())]

{'acosi_i': 1.4897526614420296516,
 'acosi_o': 91.406877920363995858,
 'asini_i': 1.2175284370901029975,
 'asini_o': 74.672709712018036338,
 'delta': 0.0,
 'delta_lan': 4.8220241628483202396e-05,
 'eps1_i': 0.0006856734568106118968,
 'eps1_o': 0.035186277265651086594,
 'eps2_i': -9.1706450063988849455e-05,
 'eps2_o': -0.0034621368067846675071,
 'f0': 365.95336876828094,
 'j_AO1350': 5.3635190284990956237e-05,
 'j_AO1440': 4.9259452945711710099e-05,
 'j_AO327': 6.4794561335151349273e-05,
 'j_GBT1500': 6.2620813306723303735e-05,
 'j_GBT350': 1.8855899287062338046e-05,
 'j_GBT820': 6.7057259469771009361e-05,
 'j_WSRT350': -3.6104142237134380419e-05,
 'pb_i': 1.6294017912103802948,
 'pb_o': 327.25753790301990501,
 'ppn_mode': None,
 'q_i': 0.13734741974750966142,
 'shapiro': True,
 'tasc_i': 0.40771744400081470636,
 'tasc_o': 313.93581137879523948,
 'tol': 1e-16,
 'use_quad': False}

In [15]:
for ot in [['AO1350','AO1440'],['GBT1500'],['WSRT1400']]:
    args = F.args.copy()
    args['only_tels'] = tuple(ot)
    bpd[frozenset(args.iteritems())] = bpd[frozenset(F.args.iteritems())]

## Fill in missing parameter values